In [1]:
""" External Imports """
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import sys
import yaml
import numpy as np
sys.path.append("/Users/emilieyu/endotehelial-masboss")

ipylab module is not installed, menus and toolbar are disabled.


In [2]:
""" Packge Imports """
from boolean_models.analysis import (
    compute_delta,
    classify_phenotype,
    save_df_to_csv, 
    generate_ko_model
)
from boolean_models.scripts.run_perturbations import run_perturbations
from boolean_models.scripts.run_param_sweep import *


In [3]:

""" Global Setting Configuration """

PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"
SWEEP_CONFIG_PATH = PROJECT_ROOT / "config" / "parameter_sweep_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

with open(SWEEP_CONFIG_PATH, "r") as f:
    sweep_config = yaml.safe_load(f)

# MaBoSS Model files
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']
MODELS_CFG = PROJECT_ROOT / config['paths']['model_cfg']

# Result directory
RESULTS_DIR = PROJECT_ROOT / config['paths']['results_base']

# Subdirectories to store parameter sweep / perturbation result
PARAM_DIR = RESULTS_DIR / config['paths']['subdirs']['param_sweep']
PERBS_DIR = RESULTS_DIR / config['paths']['subdirs']['perturbation_sim']

# Perturbations
PERBS_DICT = config['perturbations'] # get mutations directly from config
PARAM_LIST = config['sensitivity_analysis'] # get list of parameters

In [4]:
base_model = maboss.load(str(MODELS_BND), str(MODELS_CFG))
base_model.param['max_time'] = 10.0
base_model.param['sample_count'] = 5000
#base_model.param

## 2D Param Sweep


In [5]:
param_values = build_ranges(sweep_config)
exp_1d_cfg = sweep_config['1D_experiments']
exp_2d_cfg = sweep_config['2D_experiments']
perb_config = config['perturbations']
exp_cfg = sweep_config['experiments']

In [ ]:
results_1d_df = run_1d_sweeps(base_model, exp_1d_cfg, perb_config, param_values)
results_2d_df = run_2d_sweeps(base_model, exp_2d_cfg, perb_config, param_values)

DEBUG: Starting 1D sweep for perturbation: WT with parameter dict_keys(['$RhoA_amp', '$RhoA_mod', '$RhoA_basal', '$RhoA_antagonistic', '$RhoA_decay', '$RhoC_amp', '$RhoC_mod', '$RhoC_basal', '$RhoC_antagonistic', '$RhoC_decay'])
DEBUG: All 1D sweeps completed
DEBUG: Starting 1D sweep for perturbation: DSP with parameter ['$RhoC_antagonistic', '$RhoC_mod', '$RhoC_amp', '$RhoC_decay']
DEBUG: All 1D sweeps completed
DEBUG: Starting 1D sweep for perturbation: TJP1 with parameter ['$RhoA_antagonistic', '$RhoA_mod', '$RhoA_amp', '$RhoA_decay']
DEBUG: All 1D sweeps completed
DEBUG: Starting 1D sweep for perturbation: JCAD with parameter ['$RhoA_mod', '$RhoC_mod', '$RhoA_basal', '$RhoC_basal']
DEBUG: All 1D sweeps completed
DEBUG: Starting 1D sweep for perturbation: DSP_JCAD with parameter ['$RhoC_mod', '$RhoC_basal', '$RhoA_basal']
DEBUG: All 1D sweeps completed
DEBUG: Starting 1D sweep for perturbation: TJP1_JCAD with parameter ['$RhoA_mod', '$RhoA_basal', '$RhoC_basal']
DEBUG: All 1D sweeps

In [11]:
results_1d_df

,DSP,JCAD,RhoA,RhoC,TJP1,p_name,p_value,perb
9.9000,0.501800,0.498601,0.570033,0.604503,0.507601,$RhoA_amp,5.0,WT
9.9000,0.494801,0.509799,0.592083,0.602749,0.503599,$RhoA_amp,10.0,WT
9.9000,0.499799,0.501401,0.607544,0.599523,0.502202,$RhoA_amp,15.0,WT
9.9000,0.501601,0.488001,0.613084,0.591813,0.498802,$RhoA_amp,20.0,WT
9.9000,0.509199,0.503200,0.606527,0.591332,0.500001,$RhoA_amp,25.0,WT
...,...,...,...,...,...,...,...,...
9.9000,0.512600,NaN,0.617556,0.432992,NaN,$RhoA_basal,0.6,TJP1_JCAD
9.9000,0.496600,NaN,0.694916,0.390364,NaN,$RhoA_basal,1.1,TJP1_JCAD
9.9000,0.503800,NaN,0.817813,0.084283,NaN,$RhoC_basal,0.1,TJP1_JCAD
9.9000,0.504001,NaN,0.668396,0.364647,NaN,$RhoC_basal,0.6,TJP1_JCAD


In [16]:
results_2d_df['delta'] = compute_delta(results_2d_df, config)
results_2d_df['phenotype'] = results_2d_df['delta'].apply(lambda x: classify_phenotype(x, config))
results_2d_df

,DSP,JCAD,RhoA,RhoC,TJP1,$RhoA_antagonistic,$RhoC_antagonistic,perb,exp_name,$RhoA_amp,delta,phenotype
9.9000,0.502600,0.498600,0.702323,0.710107,0.505800,1.0,1.0,WT,Competition_Landscape,NaN,0.007784,Normal
9.9000,0.497799,0.506599,0.733386,0.587784,0.495599,1.0,2.0,WT,Competition_Landscape,NaN,-0.145602,Normal
9.9000,0.504401,0.498801,0.750565,0.522293,0.500802,1.0,3.0,WT,Competition_Landscape,NaN,-0.228272,Normal
9.9000,0.506798,0.493200,0.770058,0.468711,0.499198,1.0,4.0,WT,Competition_Landscape,NaN,-0.301347,Failed
9.9000,0.501001,0.495600,0.780700,0.433480,0.503800,1.0,5.0,WT,Competition_Landscape,NaN,-0.347220,Failed
...,...,...,...,...,...,...,...,...,...,...,...,...
9.9000,0.494601,0.510001,0.540656,0.504603,NaN,10.0,NaN,TJP1,Escape_Threshold_RhoA,55.0,-0.036053,Normal
9.9000,0.495001,0.493002,0.511257,0.524927,NaN,11.0,NaN,TJP1,Escape_Threshold_RhoA,55.0,0.013670,Normal
9.9000,0.499398,0.493999,0.505790,0.524910,NaN,12.0,NaN,TJP1,Escape_Threshold_RhoA,55.0,0.019120,Normal
9.9000,0.491400,0.483999,0.483466,0.545943,NaN,13.0,NaN,TJP1,Escape_Threshold_RhoA,55.0,0.062477,Normal


In [18]:
long_df = results_2d_df.melt(
    id_vars=["perb", "exp_name", "phenotype", "delta"],
    var_name="parameter",
    value_name="p_value"
)
long_df

,perb,exp_name,phenotype,delta,parameter,p_value
0,WT,Competition_Landscape,Normal,0.007784,DSP,0.502600
1,WT,Competition_Landscape,Normal,-0.145602,DSP,0.497799
2,WT,Competition_Landscape,Normal,-0.228272,DSP,0.504401
3,WT,Competition_Landscape,Failed,-0.301347,DSP,0.506798
4,WT,Competition_Landscape,Failed,-0.347220,DSP,0.501001
...,...,...,...,...,...,...
4027,TJP1,Escape_Threshold_RhoA,Normal,-0.036053,$RhoA_amp,55.000000
4028,TJP1,Escape_Threshold_RhoA,Normal,0.013670,$RhoA_amp,55.000000
4029,TJP1,Escape_Threshold_RhoA,Normal,0.019120,$RhoA_amp,55.000000
4030,TJP1,Escape_Threshold_RhoA,Normal,0.062477,$RhoA_amp,55.000000


In [ ]:
def run_1d_sweeps(params, perb_model, perb, exp_name):
    exp_results = []
    for p in params: 
        values = param_values[p]
        for v in values:
            m_temp = perb_model.copy()
            m_temp.update_parameters(**{p: v})

            res = m_temp.run()
            ss_df = res.get_last_nodes_probtraj()

            ss_df[['p_name', 'p_value', 'perb', 'exp_name']] = pd.DataFrame([[p, v, perb, exp_name]], index=ss_df.index)

            exp_results.append(ss_df)
            
    return pd.concat(exp_results)

In [ ]:
def run_sweeps(base_model, exp_config, perb_config, param_values):
    """
    Handles both 1D and 2D experiments correctly:
    - 1D → single-parameter sweep
    - 2D → cartesian product sweep
    """

    results = []

    for exp in exp_config:
        exp_name = exp["name"]
        perbs = exp["perturbations"]
        params = exp["parameters"]
        type = exp['type']

        print(exp_name)
        for perb in perbs:
            print(type)
            print(perb)
            perb_model = generate_ko_model(base_model, perb_config[perb])

            # Resolve "all" for 1D case
            if params == "all":
                sweep_params = list(param_values.keys())
            else:
                sweep_params = params

            # --------------------------------------------------
            # 1D Sweep (NO cross product)
            # --------------------------------------------------
            if type == '1D':
                p = sweep_params[0]
                values = param_values[p]

                for v in values:
                    m_temp = perb_model.copy()
                    m_temp.update_parameters(**{p: v})

                    res = m_temp.run()
                    ss_df = res.get_last_nodes_probtraj()

                    ss_df[p] = v
                    ss_df["perb"] = perb
                    ss_df["exp_name"] = exp_name
                    ss_df["dim"] = type

                    results.append(ss_df)

            # --------------------------------------------------
            # 2D Sweep (Cartesian product)
            # --------------------------------------------------
            elif type == '2D':
                p1, p2 = sweep_params
                v1_list = param_values[p1]
                v2_list = param_values[p2]

                for v1, v2 in itertools.product(v1_list, v2_list):
                    m_temp = perb_model.copy()
                    m_temp.update_parameters(**{p1: v1, p2: v2})

                    res = m_temp.run()
                    ss_df = res.get_last_nodes_probtraj()

                    ss_df[p1] = v1
                    ss_df[p2] = v2
                    ss_df["perb"] = perb
                    ss_df["exp_name"] = exp_name
                    ss_df["dim"] = type

                    results.append(ss_df)

            else:
                raise ValueError("Only 1D and 2D experiments supported.")

    return pd.concat(results, ignore_index=True)


In [20]:
df = run_sweeps(base_model, exp_cfg, perb_config, param_values)

WT_Sweep
1D
WT
DSP_Sweep
1D
DSP
TJP1_Sweep
1D
TJP1
JCAD_Sweep
1D
JCAD
DSP_JCAD_Sweep
1D
DSP_JCAD
TJP1_JCAD_Sweep
1D
TJP1_JCAD
Competition_Landscape
2D
WT
Escape_Threshold_RhoA
2D
WT
2D
TJP1


In [21]:
df

,DSP,JCAD,RhoA,RhoC,TJP1,$RhoA_amp,perb,exp_name,dim,$RhoC_antagonistic,$RhoA_antagonistic,$RhoA_mod,$RhoC_mod
0,0.505598,0.496398,0.563405,0.599639,0.493199,5.0,WT,WT_Sweep,1,NaN,NaN,NaN,NaN
1,0.497599,0.488400,0.592491,0.589147,0.500001,10.0,WT,WT_Sweep,1,NaN,NaN,NaN,NaN
2,0.513599,0.501199,0.611861,0.600512,0.508799,15.0,WT,WT_Sweep,1,NaN,NaN,NaN,NaN
3,0.496000,0.504598,0.612992,0.589955,0.486399,20.0,WT,WT_Sweep,1,NaN,NaN,NaN,NaN
4,0.500000,0.495997,0.624132,0.590224,0.494799,25.0,WT,WT_Sweep,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.500600,0.505999,0.535514,0.514333,NaN,55.0,TJP1,Escape_Threshold_RhoA,2,NaN,10.0,NaN,NaN
560,0.501798,0.510600,0.529932,0.513935,NaN,55.0,TJP1,Escape_Threshold_RhoA,2,NaN,11.0,NaN,NaN
561,0.499999,0.500798,0.515656,0.522691,NaN,55.0,TJP1,Escape_Threshold_RhoA,2,NaN,12.0,NaN,NaN
562,0.486602,0.514401,0.493703,0.530447,NaN,55.0,TJP1,Escape_Threshold_RhoA,2,NaN,13.0,NaN,NaN


In [ ]:
long_df = df.melt(
    id_vars=["perb", "exp_name", "dim", "node"],
    var_name="parameter",
    value_name="value"
)
long_df

In [44]:
exp_config = sweep_config['1D_experiments']
exp_config
result_1d = []
for perb, params in exp_config.items():
    m = generate_ko_model(base_model, perb_config[perb])
    if params == 'all': 
        params = param_values.keys()
    for p in params:
        values = param_values[p]
        for v in values: 
            m_temp = m.copy()
            m_temp.update_parameters(**{p: v})
            res = m_temp.run()
            ss_df = res.get_last_nodes_probtraj()
            ss_df[['p_name', 'p_value', 'perb']] = pd.DataFrame([[p, v, perb]], index=ss_df.index)

            result_1d.append(ss_df)


In [45]:
result_1d_df = pd.concat(result_1d)
result_1d_df

,DSP,JCAD,RhoA,RhoC,TJP1,p_name,p_value,perb
9.9000,0.492397,0.498199,0.564916,0.612862,0.500798,$RhoA_amp,5.0,WT
9.9000,0.506200,0.502800,0.605704,0.595145,0.515201,$RhoA_amp,10.0,WT
9.9000,0.505401,0.496002,0.611292,0.591205,0.506402,$RhoA_amp,15.0,WT
9.9000,0.512402,0.502401,0.608345,0.599146,0.500001,$RhoA_amp,20.0,WT
9.9000,0.511001,0.498001,0.625592,0.596013,0.514401,$RhoA_amp,25.0,WT
...,...,...,...,...,...,...,...,...
9.9000,0.506200,NaN,0.613428,0.444897,NaN,$RhoA_basal,0.6,TJP1_JCAD
9.9000,0.494200,NaN,0.690220,0.402260,NaN,$RhoA_basal,1.1,TJP1_JCAD
9.9000,0.503799,NaN,0.812894,0.085680,NaN,$RhoC_basal,0.1,TJP1_JCAD
9.9000,0.497600,NaN,0.690379,0.346808,NaN,$RhoC_basal,0.6,TJP1_JCAD


In [49]:
result_1d_df['delta'] = compute_delta(result_1d_df, config)
result_1d_df['phenotype'] = result_1d_df['delta'].apply(lambda x: classify_phenotype(x, config))
result_1d_df[result_1d_df['perb'] == 'TJP1_JCAD']

,DSP,JCAD,RhoA,RhoC,TJP1,p_name,p_value,perb,delta,phenotype
9.9000,0.506800,NaN,0.529037,0.490322,NaN,$RhoA_mod,1.0,TJP1_JCAD,-0.038715,Normal
9.9000,0.499001,NaN,0.613666,0.447575,NaN,$RhoA_mod,2.0,TJP1_JCAD,-0.166091,Normal
9.9000,0.508800,NaN,0.649740,0.420092,NaN,$RhoA_mod,3.0,TJP1_JCAD,-0.229648,Normal
9.9000,0.492401,NaN,0.678606,0.394870,NaN,$RhoA_mod,4.0,TJP1_JCAD,-0.283736,Failed
9.9000,0.491601,NaN,0.674048,0.401106,NaN,$RhoA_mod,5.0,TJP1_JCAD,-0.272942,Failed
9.9000,0.500000,NaN,0.697693,0.388297,NaN,$RhoA_mod,6.0,TJP1_JCAD,-0.309396,Failed
9.9000,0.495199,NaN,0.695644,0.395046,NaN,$RhoA_mod,7.0,TJP1_JCAD,-0.300598,Failed
9.9000,0.497800,NaN,0.454376,0.540191,NaN,$RhoA_basal,0.1,TJP1_JCAD,0.085815,Normal
9.9000,0.506200,NaN,0.613428,0.444897,NaN,$RhoA_basal,0.6,TJP1_JCAD,-0.168531,Normal
9.9000,0.494200,NaN,0.690220,0.402260,NaN,$RhoA_basal,1.1,TJP1_JCAD,-0.287960,Failed


## Run Perturbations

In [5]:
perb_df = run_perturbations(base_model, PERBS_DIR, config)
perb_df

DEBUG: Running perturbation: WT
DEBUG: Running perturbation: DSP
DEBUG: Running perturbation: TJP1
DEBUG: Running perturbation: JCAD
DEBUG: Running perturbation: DSP_JCAD
DEBUG: Running perturbation: TJP1_JCAD
DEBUG: All simulations completed successfully
DEBUG: File perturbation_timeseries_2026-02-16_15-02-01.csv successfully written to directory: /Users/emilieyu/endotehelial-masboss/results/boolean_models/exploratory/perturbation_sim


,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
0,0.0,0.504400,0.500801,0.501398,0.537655,0.537090,WT,-0.000565,Normal
1,0.1,0.504399,0.500800,0.501399,0.569181,0.578226,WT,0.009045,Normal
2,0.2,0.504400,0.500798,0.501401,0.580498,0.592461,WT,0.011963,Normal
3,0.3,0.504400,0.500799,0.501400,0.581366,0.597023,WT,0.015657,Normal
4,0.4,0.504401,0.500800,0.501400,0.577839,0.596077,WT,0.018238,Normal
...,...,...,...,...,...,...,...,...,...
595,9.5,0.499800,0.000000,0.000000,0.637157,0.421043,TJP1_JCAD,-0.216114,Normal
596,9.6,0.499801,0.000000,0.000000,0.642515,0.422537,TJP1_JCAD,-0.219978,Normal
597,9.7,0.499799,0.000000,0.000000,0.639263,0.422745,TJP1_JCAD,-0.216518,Normal
598,9.8,0.499800,0.000000,0.000000,0.634185,0.423781,TJP1_JCAD,-0.210404,Normal


In [6]:
dsp_df = perb_df[perb_df['perturbation'] == 'DSP_JCAD']
dsp_df

,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
400,0.0,0.0,0.491600,0.0,0.487320,0.500907,DSP_JCAD,0.013587,Normal
401,0.1,0.0,0.491600,0.0,0.462228,0.519936,DSP_JCAD,0.057708,Normal
402,0.2,0.0,0.491600,0.0,0.450690,0.545560,DSP_JCAD,0.094870,Normal
403,0.3,0.0,0.491601,0.0,0.445165,0.559664,DSP_JCAD,0.114499,Normal
404,0.4,0.0,0.491600,0.0,0.442231,0.579204,DSP_JCAD,0.136973,Normal
...,...,...,...,...,...,...,...,...,...
495,9.5,0.0,0.491600,0.0,0.425271,0.639371,DSP_JCAD,0.214100,Normal
496,9.6,0.0,0.491600,0.0,0.418016,0.640043,DSP_JCAD,0.222027,Normal
497,9.7,0.0,0.491600,0.0,0.420439,0.644857,DSP_JCAD,0.224418,Normal
498,9.8,0.0,0.491600,0.0,0.424778,0.643886,DSP_JCAD,0.219108,Normal


## Run Sensitivity Analysis

In [5]:
wt_param_df = run_param_sweep_single(base_model, PARAM_DIR, config, perturbation='WT')
wt_param_df

DEGUG: Performing sweep for parameter $RhoA_amp with values: [ 5. 10. 15. 20. 25. 30. 35. 40. 45. 50. 55.]
DEGUG: Performing sweep for parameter $RhoA_mod with values: [1. 3. 5. 7.]
DEGUG: Performing sweep for parameter $RhoA_basal with values: [0.1 0.6 1.1]
DEGUG: Performing sweep for parameter $RhoA_antagonistic with values: [ 1.  3.  5.  7.  9. 11. 13.]
DEGUG: Performing sweep for parameter $RhoA_decay with values: [0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
DEGUG: Performing sweep for parameter $RhoC_amp with values: [ 5. 10. 15. 20. 25. 30. 35. 40. 45. 50. 55.]
DEGUG: Performing sweep for parameter $RhoC_mod with values: [1. 3. 5. 7.]
DEGUG: Performing sweep for parameter $RhoC_basal with values: [0.1 0.6 1.1]
DEGUG: Performing sweep for parameter $RhoC_antagonistic with values: [ 1.  3.  5.  7.  9. 11. 13.]
DEGUG: Performing sweep for parameter $RhoC_decay with values: [0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
DEBUG: Sweep of parameters for WT completed. 
DEBUG:

,DSP,JCAD,RhoA,RhoC,TJP1,param_value,param_name,delta,phenotype
0,0.504000,0.504200,0.578332,0.597347,0.503000,5.00,$RhoA_amp,0.019015,Normal
1,0.504401,0.496202,0.591574,0.597296,0.487602,10.00,$RhoA_amp,0.005722,Normal
2,0.504401,0.498999,0.611785,0.588377,0.495801,15.00,$RhoA_amp,-0.023408,Normal
3,0.500001,0.495201,0.609034,0.590839,0.497200,20.00,$RhoA_amp,-0.018195,Normal
4,0.508400,0.500999,0.625612,0.579402,0.501399,25.00,$RhoA_amp,-0.046210,Normal
...,...,...,...,...,...,...,...,...,...
65,0.500402,0.500401,0.616113,0.549282,0.492002,0.55,$RhoC_decay,-0.066831,Normal
66,0.505201,0.485600,0.601922,0.548050,0.502400,0.65,$RhoC_decay,-0.053872,Normal
67,0.497600,0.499801,0.598165,0.536718,0.510201,0.75,$RhoC_decay,-0.061447,Normal
68,0.495801,0.513400,0.612028,0.525867,0.498200,0.85,$RhoC_decay,-0.086161,Normal


In [12]:
wt_param_df[wt_param_df['phenotype'] != ('Normal')]

,DSP,JCAD,RhoA,RhoC,TJP1,param_value,param_name,delta,phenotype
22,0.491399,0.506398,0.421069,0.691762,0.509799,9.0,$RhoA_antagonistic,0.270693,Hyper
23,0.488999,0.503000,0.384173,0.694786,0.492999,11.0,$RhoA_antagonistic,0.310613,Hyper
24,0.494800,0.503800,0.354036,0.724164,0.494199,13.0,$RhoA_antagonistic,0.370128,Hyper
50,0.504602,0.516001,0.682819,0.429712,0.493003,0.1,$RhoC_basal,-0.253107,Failed
57,0.502599,0.489800,0.698127,0.415241,0.503600,9.0,$RhoC_antagonistic,-0.282886,Failed
58,0.506802,0.508601,0.709154,0.383527,0.500603,11.0,$RhoC_antagonistic,-0.325627,Failed
59,0.500601,0.510600,0.716618,0.359586,0.513998,13.0,$RhoC_antagonistic,-0.357032,Failed


## Adapted Sensitivity Analysis

In [7]:
sweep_df = run_param_sweep(base_model, PARAM_DIR, config)

DEBUG: Running perturbation: WT
DEGUG: Starting sweep for perturbation WT
DEGUG: Performing sweep for parameter $RhoA_amp with values: [ 5.  9. 13. 17. 21. 25. 29. 33. 37. 41. 45. 49. 53. 57.]
DEGUG: Performing sweep for parameter $RhoA_mod with values: [1. 2. 3. 4. 5. 6. 7.]
DEGUG: Performing sweep for parameter $RhoA_basal with values: [0.1 0.3 0.5 0.7 0.9 1.1 1.3]
DEGUG: Performing sweep for parameter $RhoA_antagonistic with values: [1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5 9.  9.5]
DEGUG: Performing sweep for parameter $RhoA_decay with values: [0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
DEGUG: Performing sweep for parameter $RhoC_amp with values: [ 5.  9. 13. 17. 21. 25. 29. 33. 37. 41. 45. 49. 53. 57.]
DEGUG: Performing sweep for parameter $RhoC_mod with values: [1. 2. 3. 4. 5. 6. 7.]
DEGUG: Performing sweep for parameter $RhoC_basal with values: [0.1 0.3 0.5 0.7 0.9 1.1 1.3]
DEGUG: Performing sweep for parameter $RhoC_antagonistic with values: [1.  

In [11]:

sweep_df[sweep_df['perturbation'] == 'TJP1']

,DSP,JCAD,RhoA,RhoC,TJP1,param_value,param_name,perturbation,delta,phenotype
161,0.493200,0.497000,0.654388,0.419116,NaN,5.00,$RhoA_amp,TJP1,-0.235272,Normal
162,0.503400,0.514201,0.684273,0.398057,NaN,9.00,$RhoA_amp,TJP1,-0.286216,Failed
163,0.499000,0.493600,0.679784,0.391617,NaN,13.00,$RhoA_amp,TJP1,-0.288167,Failed
164,0.506600,0.509601,0.694793,0.389971,NaN,17.00,$RhoA_amp,TJP1,-0.304822,Failed
165,0.501601,0.511000,0.690249,0.391163,NaN,21.00,$RhoA_amp,TJP1,-0.299086,Failed
166,0.502799,0.495000,0.694663,0.385914,NaN,25.00,$RhoA_amp,TJP1,-0.308749,Failed
167,0.488400,0.504601,0.689921,0.400912,NaN,29.00,$RhoA_amp,TJP1,-0.289009,Failed
168,0.495598,0.499999,0.691860,0.389864,NaN,33.00,$RhoA_amp,TJP1,-0.301996,Failed
169,0.501599,0.494599,0.699586,0.391785,NaN,37.00,$RhoA_amp,TJP1,-0.307801,Failed
170,0.496400,0.490399,0.699791,0.380485,NaN,41.00,$RhoA_amp,TJP1,-0.319306,Failed


{'DSP': 'OFF'}

{'amp': {'range': [5.0, 60.0], 'step': 4.0},
 'mod': {'range': [1.0, 8.0], 'step': 1.0},
 'basal': {'range': [0.1, 1.5], 'step': 0.2},
 'antagonistic': {'range': [1.0, 10.0], 'step': 0.5},
 'decay': {'range': [0.05, 1.0], 'step': 0.1}}

dict_items([('RhoA', {'prefix': '$RhoA', 'parameters': ['amp', 'mod', 'basal', 'antagonistic', 'decay']}), ('RhoC', {'prefix': '$RhoC', 'parameters': ['amp', 'mod', 'basal', 'antagonistic', 'decay']})])